In [4]:
import keras.backend as K


def getParameters(x):

    #since x comes in as a batch with shape (20,4) -- (or any other batch size different from 20)

    #let's condense X in one sample only, because we want only 4 elements, not 20*4 elements
    xCondensed = K.mean(x,axis=0,keepdims=True)
        #I'm using keepdims because we will need that x end up with the same number of samples for compatibility purposes (keras rules)

    #let's expand x again (for compatibility purposes), now repeating the 4 values 20 (or more) times
    #return K.ones_like(x) * xCondensed
    return x



def yourFunction(x):


    #now x has 4 parameters (assuming you had a Dense(4) before these lambda layers)
    A1 = x[:,0]
    b = x[:,1]
    c = x[:,2]
    d = x[:,3]

    #creating the 20 (or more) iterations
    ones = K.ones((20,1))
    iterationsStartingAt1= K.cumsum(ones)
    iterationsStartingAt0= iterationsStartingAt1 - 1
    iterations = iterationsStartingAt0

    return A1*np.exp(-(t+toff1)/t1)+A2*np.exp(-(t+toff2)/t2)+j0
    return (a * K.sin((b*iterations) + c)) + d


In [3]:
#commentary to this specific problem found here:
#https://stackoverflow.com/questions/46227823/keras-neural-network-outputting-function-parameters-how-to-construct-loss-func

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Lambda
from keras import backend as K
from keras.utils import plot_model


inputs_parameters = tf.keras.Input(shape=(300,))
inputs_second_half = tf.keras.Input(shape=(200,))

# Build the model
model = Sequential()

# Add input layer
model.add(Dense(units=input_size, activation='relu', input_dim=input_size))

# Add hidden layers (you can customize the number of layers and units)
num_hidden_layers = 2
for _ in range(num_hidden_layers):
    model.add(Dense(units=32, activation='relu'))

# Add output layer
model.add(Dense(units=output_size, activation='linear'))

#model.add(Lambda(getParameters,output_shape=(4,),name='paramLayer'))
#model.add(Lambda(yourFunction,output_shape=(1,),name='valueLayer'))

# Compile the model with the custom loss function
model.compile(optimizer='adam')

print(model.summary())
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [16]:
import keras

import tensorflow.keras.backend as K

def custom_loss(y_true, y_pred):
    """
    Custom loss function for the specified task.

    Parameters:
    - y_true: Ground truth values, of shape (batch_size, 2, N).
    - y_pred: Predicted values, of shape (batch_size, 7).

    Returns:
    - loss: Scalar value representing the mean loss over the batch.
    """

    # Extract x and y values from y_true
    x_values = y_true[:, 0, :]  # Shape: (batch_size, N)
    y_values = y_true[:, 1, :]  # Shape: (batch_size, N)
    print(y_pred.shape)
    

    # Apply the function to each element of y_pred based on the corresponding x_value
    # Result shape: (batch_size, N)
    function_values = (
        y_pred[:, 0, :] * K.exp(-(x_values + y_pred[:, 1, :]) / y_pred[:, 2, :]) +
        y_pred[:, 3, :] * K.exp(-(x_values + y_pred[:, 4, :]) / y_pred[:, 5, :]) +
        y_pred[:, 6, :]
    )
    #print(function_values)
    # Compute the difference between the function values and y_values
    # Result shape: (batch_size, N)
    diff = function_values - y_values

    # Take the square of the differences
    squared_diff = K.square(diff)

    # Take the mean over the batch
    # Result shape: (batch_size,)
    loss = K.mean(squared_diff, axis=-1)

    return loss

def create_nn():
    global X_data, points
    # Input layer, the number of input nodes is governed by X_data.shape[1]
    # X_data.shape[1] is the number of columns in X_data
    inputs = keras.Input(shape=(X_data.shape[1],), name='input')

    # Dense layers with relu activations
    layers_dense = keras.layers.Dense(100, 'relu')(inputs)
    # add a second dense layer with 50 hidden neurons and using the relu activation
    layers_dense2 = keras.layers.Dense(50, 'relu')(layers_dense)
    # Output layer, only one output node is used
    parameters = keras.layers.Dense(7)(layers_dense2)
    # Expand parameters to have same shape as y_true
    expanded_parameters = keras.layers.RepeatVector(points)(parameters)


    return keras.Model(inputs=inputs, outputs=expanded_parameters, name="current_function_prediction")

In [17]:
#test loss function
import numpy as np
batch_size = 32
lenght = 10


y_true = 255 * np.random.rand(batch_size, 2, lenght)
y_pred = 255 * np.random.rand(batch_size, 7, lenght)

print(custom_loss(y_true, y_pred))

#print(K.eval(custom_loss(K.variable(y_true), K.variable(y_pred))).sum())


(32, 7)


ValueError: operands could not be broadcast together with shapes (32,10) (32,) 

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)